In [117]:
import numpy as np
import json
from sklearn.feature_extraction import text

In [118]:
x = open('fedpapers_split.txt').read()
papers = json.loads(x)

In [119]:
papersH = papers[0]
papersM = papers[1]
papersD = papers[2]
nH, nM, nD = len(papersH), len(papersM), len(papersD)
print "nH = %d, nM = %d, nD = %d" % (nH, nM, nD)

nH = 51, nM = 17, nD = 12


In [120]:
stop_words = text.ENGLISH_STOP_WORDS.union({'HAMILTON','MADISON'})
vectorizer = text.CountVectorizer(stop_words,min_df=10)
X = vectorizer.fit_transform(papersH+papersM+papersD).toarray()
#vectorizer.vocabulary_

In [121]:
XH, XM, XD = X[:nH,:], X[nH:nH+nM,:], X[nH+nM:,:]
print "XH = %d X %d, XM = %d X %d, XD = %d X %d" % (len(XH),len(XH[0]),len(XM),len(XM[0]),len(XD),len(XD[0]))

XH = 51 X 1515, XM = 17 X 1515, XD = 12 X 1515


In [122]:
# Initialize vectors for P(word_j | H/M) as total occurence of a word for an other divided by total words 
fH = np.ones(len(XH[0]))
totH = len(XH[0])
fM = np.ones(len(XM[0]))
totM = len(XM[0])

for i in range(0,len(XH[0])):
    for j in range(0,len(XH)):
        fH[i] = float(fH[i])+XH[j][i]
    totH = totH + fH[i]
fH = fH/totH
print "End fH, with mean = %.6f" % fH[0]

for i in range(0,len(XM[0])):
    for j in range(0,len(XM)):
        fM[i] = fM[i]+XM[j][i]
    totM = totM + fM[i]
fM = fM/totM
print "End fM, with mean = %.6f" % fM[0]

fratio = fH/fM
print fratio

End fH, with mean = 0.000198
End fM, with mean = 0.000169
[ 1.16899041  1.03860302  4.90975972 ...,  0.92057995  1.66138334
  0.48353694]


In [123]:
# Compute prior probabilities 
piH = float(nH)/(nH+nM)
piM = float(nM)/(nH+nM)
print "piH = %.6f, piM = %.6f" % (piH, piM)

piH = 0.750000, piM = 0.250000


In [124]:
Ham_tot = nH
Mad_tot = nM
for xd in XD: # Iterate over disputed documents
    rat = 1
    # Compute likelihood ratio for Naive Bayes model
    for j in range(0,len(xd)):
        rat = rat*(fratio[j])**xd[j]
        
    LR = (piH/piM)*rat
    if LR>1:
        print 'Hamilton'
        Ham_tot=Ham_tot+1
    else:
        print 'Madison'
        Mad_tot=Mad_tot+1
print "Hamilton wrote %d total, and %d of the disputed." % (Ham_tot, Ham_tot-nH)
print "Madison wrote %d total, and %d of the disputed." % (Mad_tot, Mad_tot-nM)

Madison
Madison
Madison
Madison
Hamilton
Hamilton
Hamilton
Hamilton
Hamilton
Madison
Madison
Madison
Hamilton wrote 56 total, and 5 of the disputed.
Madison wrote 24 total, and 7 of the disputed.


In [125]:
%run fedpapers.py

Madison
Madison
Madison
Madison
Madison
Madison
Madison
Madison
Madison
Madison
Madison
Madison
Hamilton wrote 51 total, and 0 of the disputed.
Madison wrote 29 total, and 12 of the disputed.


fedpapers.py:55: RuntimeWarning: divide by zero encountered in divide
  fratio = fH/fM
fedpapers.py:67: RuntimeWarning: invalid value encountered in double_scalars
  rat = rat*(fratio[j])**xd[j]
